In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import fiona

In [4]:
# p10 geography
p10 = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\UrbanSim Data Review Option B v1c\p10_geo_shp_yq.shp')

## parcel10 to pba40 basezoning code
pz10_raw = pd.read_csv(r'C:\Users\ywang\Box\Mine\1_UrbanSim\2020_03_06_zoning_parcels.csv')

# pba40 basezoning plu
plu10 = pd.read_csv('C:/Users/ywang/Documents/GitHub/bayarea_urbansim/data/zoning_lookup.csv')

# pba50 basezoning PLU
print(fiona.listlayers(r'C:\Users\ywang\Documents\ArcGIS\Projects\UrbanSim Data Review Option B v1c\UrbanSim Data Review Option B v1c.gdb'))
p10_plu50_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\UrbanSim Data Review Option B v1c\UrbanSim Data Review Option B v1c.gdb', layer='p10_boc_opt_B_v3_geo_yq')
#plu50 = pd.read_csv(r'C:\Users\ywang\Box\Mine\3_BASIS\urbansim_boc_2010_mod.csv')

['p10_boc_opt_B_v3_geo_yq']


In [5]:
# parcel geometry
#p10 = p10_raw[['PARCEL_ID','APN','geom_id_s','COUNTY_ID','jurisdiction',
#                          'ACRES','LAND_VALUE','pda_id','zoningmodcat']]
print(p10.shape)
#display(p10.head())

# pacel to zoning code mapping
pz10 = pz10_raw[['PARCEL_ID','zoning_id','nodev_pba40']]
print(pz10.shape)
#display(pz10.head())

p10_z10 = p10.merge(pz10, on = 'PARCEL_ID', how = 'left')
print(p10_z10.shape)
#display(p10_z10.head())

# Check Number of parcels missing zoning designation
z10_missing = p10_z10.loc[p10_z10['nodev_pba40'].isnull()]
print(z10_missing.shape[0])
print(z10_missing.shape[0]/pz10.shape[0])

(1956208, 14)
(1950733, 3)
(1956208, 16)
5476
0.002807149927745109


In [6]:
## pba50 PLU

# check duplicates in zoning id
plu10 = pd.read_csv('C:/Users/ywang/Documents/GitHub/bayarea_urbansim/data/zoning_lookup.csv')
plu10['id'] = plu10['id'].apply(lambda x: float(x))
plu10['jz_o'] = plu10['city'].str.cat(plu10['name'],sep=" ")
print(plu10.shape[0], len(plu10.id.unique()), len(plu10.jz_o.unique()))

# relabel p10 land plu info (used in PBA40)
cols = [i+'_10' for i in list(plu10)]
plu10.columns = cols
#display(plu10.head())

# merge PBA40 plu to p10
p10_plu10 = p10_z10.merge(plu10, left_on = 'zoning_id', right_on = 'id_10', how = 'left')
#display(p10_plu10.head())

# Check number of p10 records failed to find a matching PLU
#display(p10_plu10.loc[p10_plu10['jz_o_10'].isnull()])
print(p10_plu10.loc[p10_plu10['jz_o_10'].isnull()].shape[0] / p10_z10.shape[0])

5156 5156 4536
0.0034745793903306807


In [7]:
## p10 w/ BASIS BOC
plu50 = p10_plu50_raw.loc[:,['PARCEL_ID','me','mt', 'mr', 'rb', 'rs', 'ih', 'iw', 'il', 'sc', 'ho', 'of', 'hm', 'ht', 'hs',
                       'max_height','max_dua','max_far','plu_id','plu_jurisdiction','plu_description']]

print(plu50.shape)

# relabel BASIS land plu info (to use in PBA50)
cols2 = [i+'_18' for i in list(plu50)]
plu50.columns = cols2
#display(plu50.head())

# merge PBA50 plu to p10
p10_plus = p10_plu10.merge(plu50, left_on = 'PARCEL_ID', right_on = 'PARCEL_ID_18', how = 'left')
print(p10_plus.shape)

#p10_plus.drop(columns = ['zoning','tablename','Unnamed: 0','id_10','name_10','plandate_10','jz_o_10','PARCEL_ID_18'],inplace = True)
p10_plus.columns = [x.upper() for x in p10_plus.columns]
#display(p10_plus.head())

(1956208, 21)
(1956208, 61)


In [8]:
for i in ['HS','HT','HM','OF','HO','SC','IL','IW','IH','RS','RB','MR','MT','ME']:
    plu = i+'_10'
    boc = i+'_18'
    p10_plus[i+'_chk'] = np.where((p10_plus[plu] == 1) & (p10_plus[boc] == 1),'both allow',np.where(
        (p10_plus[plu] == 1) & (p10_plus[boc] == 0),'only pba40 allow',np.where(
        (p10_plus[plu] == 0) & (p10_plus[boc] == 1),'only basis allow',np.where(
        (p10_plus[plu] == 0) & (p10_plus[boc] == 0),'both not allow',np.where(
        (p10_plus[plu].notnull()) & (p10_plus[boc].isnull()),'missing BASIS BOC','other')))))

In [9]:
ls_new = ['PARCEL_ID','PLU_ID_18', 'PLU_JURISDICTION_18','PLU_DESCRIPTION_18'] + [
    x + '_chk' for x in ['HS','HT','HM','OF','HO','SC','IL','IW','IH','RS','RB','MR','MT','ME']]
p10_plus_map = p10_plus[ls_new]
p10_plus_map.to_csv('p10_plus_map.csv')

In [10]:
dfs = []
dfs_diff = []
ls2 = [x + '_chk' for x in ['HS','HT','HM','OF','HO','SC','IL','IW','IH','RS','RB','MR','MT','ME']]
for i in ls2:
    print(i)
    df = p10_plus[['PARCEL_ID','PLU_ID_18','PLU_DESCRIPTION_18'] + [i]]
    df1 = p10_plus[['PARCEL_ID','PLU_ID_18','PLU_DESCRIPTION_18','GEOMETRY'] + [i]]
    df_diff = df1.loc[(df1[i] == 'only pba40 allow') | (df1[i] == 'only basis allow') | (df1[i] == 'missing BASIS BOC')]
    dfs.append(df)
    dfs_diff.append(df_diff)
    df.to_csv(i+'.csv',index = False)
    df_diff_geo = gpd.GeoDataFrame(df_diff, geometry='GEOMETRY')
    df_diff.to_csv(i + '_diff.csv',index = False)
    df_diff_geo.to_file(i + 'diff_geo.shp')

HS_chk
HT_chk
HM_chk
OF_chk
HO_chk
SC_chk
IL_chk
IW_chk
IH_chk
RS_chk
RB_chk
MR_chk
MT_chk
ME_chk


In [11]:
ls_geo = ['PARCEL_ID','GEOMETRY','PLU_ID_18', 'PLU_JURISDICTION_18','PLU_DESCRIPTION_18'] + [
    x + '_chk' for x in ['HS','HT','HM','OF','HO','SC','IL','IW','IH','RS','RB','MR','MT','ME']]
p10_plus_geo = p10_plus[ls_geo]
gdf = gpd.GeoDataFrame(p10_plus_geo, geometry='GEOMETRY')
gdf.to_file('p10_plus_map.shp')

In [ ]:
#import matplotlib.pyplot as plt
#fig, ax = plt.subplots(1, figsize=(10, 6))
#gdf.plot(column='MR_chk', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')

In [17]:
#MR = gdf[['PARCEL_ID','GEOMETRY','PLU_ID_18', 'PLU_JURISDICTION_18','PLU_DESCRIPTION_18','MR_chk']]
#MR_diff = MR.loc[(MR.MR_chk == 'only pba40 allow') | (MR.MR_chk == 'only basis allow') | (MR.MR_chk == 'missing BASIS BOC')]
#MR_diff.to_file('MR_chk_test.shp')
#MR_diff.shape

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().